# Benchmarking Automation Speech Recognition Models

## Kaggle Specific

In [ ]:
!pip install --quiet --upgrade transformers datasets accelerate timm datasets[audio]

In [ ]:
!pip install --quiet evaluate jiwer mistral-common

In [ ]:
# Setup ugging Face
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

login(token=HF_TOKEN)

## Dependencies

In [ ]:
import json
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from evaluate import load
from torchinfo import summary
from tqdm.auto import tqdm
from transformers import AutoProcessor, Gemma3nForConditionalGeneration

## Utilities

In [ ]:
main_path = Path("/kaggle/working")

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # normalize whitespace
    text = text.strip()

    return text


@torch.inference_mode()
def predict_gemma(model, processor, sample):
    result = {
        "id": sample["id"],
        "reference": normalize_text(sample["text"]),
    }

    audio = sample["audio"]
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an assistant that transcribes speech accurately.",
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": audio["array"]},
                {"type": "text", "text": "Please transcribe this audio."},
            ],
        },
    ]

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device, dtype=torch.bfloat16)
    input_len = inputs.input_ids.shape[1]

    outputs = model.generate(**inputs, do_sample=False)
    predicted_ids = outputs[0, input_len:]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)

    result["prediction"] = transcription.lower()

    return result

In [ ]:
def benchmark(model, processor, predict_fn, dataset, max_samples: int | None = None):
    benchmark_results = []
    wer = load("wer")

    for i, sample in enumerate(tqdm(dataset, desc="Benchmarking", total=max_samples)):
        if max_samples is not None and i >= max_samples:
            break

        start_time = time.perf_counter()
        prediction = predict_fn(model, processor, sample)

        end_time = time.perf_counter()

        inference_time = end_time - start_time
        wer_result = 100 * wer.compute(
            references=[prediction["reference"]],
            predictions=[prediction["prediction"]],
        )

        benchmark_results.append(
            {**prediction, "inference_time": inference_time, "wer": wer_result}
        )

    total_samples = len(benchmark_results)
    average_inference_time = np.mean(
        [result["inference_time"] for result in benchmark_results]
    ).item()
    average_wer = np.mean([result["wer"] for result in benchmark_results]).item()

    benchmark_summary = {
        "total_samples": total_samples,
        "average_inference_time": average_inference_time,
        "average_wer": average_wer,
    }

    result = {
        "benchmark_results": benchmark_results,
        "benchmark_summary": benchmark_summary,
    }

    return result

In [ ]:
def save_results(result, folder_path: Path):
    benchmark_results = pd.DataFrame(result["benchmark_results"])
    benchmark_summary = result["benchmark_summary"]

    os.makedirs(folder_path, exist_ok=True)

    benchmark_results.to_csv(folder_path / "benchmark_results.csv", index=False)

    with open(folder_path / "benchmark_summary.json", "w") as f:
        json.dump(benchmark_summary, f, indent=4)

## Dataset

In [ ]:
dataset = load_dataset("openslr/librispeech_asr", "clean", split="test", streaming=True)

## Models

### google/gemma-3n-E2B-it

In [ ]:
gemma3n_processor = AutoProcessor.from_pretrained("google/gemma-3n-e2b-it")
gemma3n_model = Gemma3nForConditionalGeneration.from_pretrained(
    "google/gemma-3n-e2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

In [ ]:
summary(gemma3n_model)

In [ ]:
!nvidia-smi

In [ ]:
results = benchmark(
    gemma3n_model,
    gemma3n_processor,
    predict_gemma,
    dataset,
    max_samples=10,
)
output_folder = main_path / "google/gemma-3n-e2b-it"
save_results(results, output_folder)